# IMPORT LIBRARIES 

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
pip install tqdm

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from tqdm import tqdm 

In [ ]:
import os
from math import erf
from matplotlib.colors import LogNorm
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rc
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
try:
    import astropy
    from astropy.table import Table, join, vstack
    from astropy.table import hstack
    from astropy.io import fits
    from astropy import units as u
    import astropy.coordinates as coord
    from astropy.coordinates import SkyCoord
    from astropy.table import Table, Column
except ImportError:
    print("astropy is required!")
finally:
    print("astropy version: %s" % astropy.__version__)

# IMPORT TRAINING AND TEST SET

In [ ]:
train_set_df_stars_GG2M_model = pd.read_csv('train_set_df_stars_GG2M_model.csv')
test_set_df_stars_GG2M_model = pd.read_csv('test_set_df_stars_GG2M_model.csv')

In [ ]:
X_train = train_set_df_stars_GG2M_model.iloc[:, 3:10]
TEFF_train = train_set_df_stars_GG2M_model.iloc[:,1]
X_test = test_set_df_stars_GG2M_model.iloc[:, 3:10]
TEFF_test = test_set_df_stars_GG2M_model.iloc[:,1]

# MODEL TRAINING

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))  
model.compile(optimizer='adam', loss='mae')


kf = KFold(n_splits=10, shuffle=True, random_state=42)

train_losses = []  
val_losses = []    

for train_index, val_index in kf.split(X_train_scaled):
    X_train_kf, X_val_kf = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_kf, y_val_kf = TEFF_train.iloc[train_index], TEFF_train.iloc[val_index]

    h = model.fit(X_train_kf, y_train_kf, validation_data=(X_val_kf, y_val_kf), epochs=50, batch_size=8)
    
    train_losses.extend(h.history['loss'])
    val_losses.extend(h.history['val_loss'])

# PREDICTIONS ON THE TEST SET

In [ ]:
DNN_pred_test = model.predict(X_test_scaled)
df_pred_DNN_test = test_set_df_stars_GG2M_model.iloc[:, :2].copy()
df_pred_DNN_test['TEFF_pred'] = DNN_pred_test
df_pred_DNN_test.to_csv('df_pred_DNN_test.csv', index=False)

# VISUALIZATION OF THE TEMPERATURE PREDICTIONS

In [ ]:
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

plt.rc('axes', labelsize=14)

rc('font', family='Times New Roman')
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'

x_DNN = TEFF_test
y_DNN = DNN_pred_test[:, 0]
residuals_DNN = x_DNN - y_DNN

media_res_DNN = np.mean(residuals_DNN)
rms_res_DNN = np.std(residuals_DNN)
median_res_DNN = np.nanmedian(residuals_DNN)

differenze_DNN = np.abs(residuals_DNN - median_res_DNN)
mad_res_DNN = np.nanmedian(differenze_DNN)
n_res_DNN = np.count_nonzero(~np.isnan(residuals_DNN))

mae_teff_DNN = mean_absolute_error(x_DNN, y_DNN)
rmse_teff_DNN = mean_squared_error(x_DNN, y_DNN, squared=False)

In [ ]:
fig = plt.figure(figsize=(7, 6))

rect1 = [0.2, 0.4, 0.65, 0.5] 
ax1 = fig.add_axes(rect1)

hb = ax1.hexbin(x_DNN, y_DNN, gridsize=120, cmap='viridis', mincnt=1)
cb = fig.colorbar(hb, label='Density')

ax1.set_ylabel(r'$T_{\mathrm{eff}}^{\mathrm{NN}}$ [K]')
ax1.tick_params(axis='both', which='both', direction='in')
ax1.minorticks_on()
ax1.plot([2000, 8500], [2000, 8500], color='k', linestyle='-')  # Linea y=x rossa

ax1.text(0.65, 0.10, r'MAE = {:.4f}'.format(mae_teff_DNN), transform=ax1.transAxes, fontsize=12)
ax1.text(0.65, 0.05, r'RMSE = {:.4f}'.format(rmse_teff_DNN), transform=ax1.transAxes, fontsize=12)

ax1.text(0.05, 0.95, r'residuals statistics:', transform=ax1.transAxes, fontsize=12)
ax1.text(0.05, 0.9, r'$\mu$ = {:.4f}'.format(media_res_DNN), transform=ax1.transAxes, fontsize=12)
ax1.text(0.05, 0.85, r'$\sigma$ = {:.4f}'.format(rms_res_DNN), transform=ax1.transAxes, fontsize=12)
ax1.text(0.05, 0.8, r'median = {:.4f}'.format(median_res_DNN), transform=ax1.transAxes, fontsize=12)
ax1.text(0.05, 0.75, r'MAD = {:.4f}'.format(mad_res_DNN), transform=ax1.transAxes, fontsize=12)
ax1.text(0.05, 0.7, r'N = {:d}'.format(n_res_DNN), transform=ax1.transAxes, fontsize=12)

rect2 = [0.2, 0.1, 0.52, 0.265] 
ax2 = fig.add_axes(rect2)

hb2 = ax2.hexbin(x_DNN, residuals_DNN, gridsize=120, cmap='viridis', mincnt=1)
ax2.plot([2000,8500], [0,0],  color='k', alpha=1, linewidth=0.8) 
ax2.set_xlabel(r'$T_{\mathrm{eff}}^{\mathrm{GES}}$ [K]')
ax2.set_ylabel('$\Delta T_{\mathrm{eff}}$ [K]')
ax2.tick_params(axis='both', which='both', direction='in')
ax2.minorticks_on()

rect3 = [0.72, 0.1, 0.2, 0.265] 
ax3 = fig.add_axes(rect3) 
xx1 = residuals_DNN.min()
xxn = residuals_DNN.max()
delta_xx = 100
ax3.hist(residuals_DNN, bins=np.arange(xx1, xxn + delta_xx, delta_xx), orientation='horizontal', edgecolor='k')
ax3.set_yticklabels([])
ax3.tick_params(axis='both', which='both', direction='in')
ax3.minorticks_on()
ax3.set_xlabel('N')
plt.savefig('TEFF_pred_DNN.png')
plt.tight_layout()

# BOOTSTRAP PROCEDURE 

In [ ]:
def build_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))  
    model.compile(optimizer='adam', loss='mae')
    return model

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

n_bootstrap = 100
train_predictions_df_new = pd.DataFrame(index=range(len(X_train_scaled)))
test_predictions_df_new = pd.DataFrame(index=range(len(X_test_scaled)))

In [ ]:
model_main = build_model()
model_main.fit(X_train_scaled, TEFF_train, epochs=50, batch_size=8, callbacks=[early_stopping])

In [ ]:
for i in tqdm(range(n_bootstrap), desc="Bootstrap Progress"):
    
    bootstrap_indices = np.random.choice(X_train_scaled.shape[0], size=X_train_scaled.shape[0], replace=True)
    X_train_bootstrap = X_train_scaled[bootstrap_indices]
    y_train_bootstrap = TEFF_train.iloc[bootstrap_indices]

    model = build_model()

    model.fit(X_train_bootstrap, y_train_bootstrap, epochs=10, batch_size=8, verbose=0, callbacks=[early_stopping])

    train_prediction_new2 = model.predict(X_train_scaled)
    train_predictions_df_new2[f"Iteration_{i}"] = train_prediction_new2.flatten()

    test_prediction_new2 = model.predict(X_test_scaled)
    test_predictions_df_new2[f"Iteration_{i}"] = test_prediction_new2.flatten()

    train_predictions_df_new2.to_csv('train_predictions_new2.csv', index=False)
    test_predictions_df_new2.to_csv('test_predictions_new2.csv', index=False)

In [ ]:
train_std_error_new = train_predictions_df_new.apply(np.std, axis=1)
test_std_error_new = test_predictions_df_new.apply(np.std, axis=1)

In [ ]:
train_IC_new = pd.DataFrame({
    'TEFF_train': TEFF_train,
})

train_IC_new['TEFF_DNN'] = df_pred_DNN_train.iloc[:, -1]
train_IC_new['std_dev'] = train_std_error_new
train_IC_new['ID'] = df_pred_DNN_train.iloc[:, 0]

In [ ]:
test_IC_new = pd.DataFrame({
    'TEFF_test': TEFF_test,
})

test_IC_new['TEFF_DNN'] = df_pred_DNN_test.iloc[:, -1]
test_IC_new['std_dev'] = test_std_error_new
test_IC_new['ID'] = df_pred_DNN_test.iloc[:, 0]

In [ ]:
train_IC_new.to_csv('train_IC_new.csv', index=False)
test_IC_new.to_csv('test_IC_new.csv', index=False)

# VISUALIZATION OF THE STD DEV PREDICTIONS

In [ ]:
fig = plt.figure(figsize=(7, 6))
# Definisci le coordinate per il rettangolo del primo set di assi [x0, y0, larghezza, altezza]
rect1 = [0.25, 0.35, 0.7, 0.55] # Grafico superiore

# Aggiungi il primo set di assi personalizzati alla figura
ax1 = fig.add_axes(rect1)

hb = ax1.hexbin(test_IC_new['TEFF_DNN'], test_IC_new['std_dev'], gridsize=120, cmap='gist_ncar_r', mincnt=1)
cb = fig.colorbar(hb, label='Density')
#
ax1.set_ylabel(r'$\sigma T_{\mathrm{eff}}^{\mathrm{NN}}$ [K]')
ax1.set_xlabel(r'$T_{\mathrm{eff}}^{\mathrm{NN}}$ [K]')
ax1.tick_params(axis='both', which='both', direction='in')
ax1.set_xlim(2000, 8500)
ax1.set_ylim(0, 600)
ax1.minorticks_on()
plt.savefig('std.dev_vs_TeffDNN_test_new.png')